# SVC - C-Support Vector Classification

## Introdução

Este notebook visa fazer a otimização de **hiperparâmetros**, de estratégia de **normalização**, de **seleção de atributos** e de **redução de dimensionalidade**. A seleção de atributos é realizada por *Sequential Feature Selection* (**SFS**, ou Seleção Sequencial de Atributos em português) por remoção de atributo, a otimização testa a ausência ou não dessa estratégia. A redução de dimensionalidade é realizada por *Principal Component Analysis* (**PCA**, ou Análise de Componenetes Principais em português), é testada sua ausência também. Para normalização, são testadas a ausência, ou a normalização **padrão** ou a pelos **mínimos e máximos**. A otimização é feita usando o módulo **optuna**.

### Modelo

O algoritmo deste notebook é o Support Vector Machine (SVM) para Classificação (ou seja, SVC). O seu método de segregação de dados pelo corte por hiperplanos possui muitas possibilidades geométricas e de proporção, assim, quando há muitos atributos e não se sabe se apresentam algum padrão de distribuição multidimensional possível, saber o kernel utilizado que melhor se adeque pode ser um problema de tentativa e erro.

Os hiperparâmetros a serem otimizados são:
* `C`: coeficiente da relação entre a margem da transição do hiperplano e a quantidade de objetos não bem separados - um "C" menor aumenta a margem e pode abranger objetos em si, de forma que, ao custo de poder errar mais, generaliza-se melhor;
* `gamma`: define a influência individual de um objeto sobre o encaixe ("fitting") do hiperplano (para sigmoid, poly e RBF);
* `kernel`: método de traçagem do plano, utilizaremos os 4 citados;
* `coef0`: termo independente do kernel polinomial e do sigmoidal;
* `degree`: grau do kernel polinomial.

In [1]:
from pandas import read_csv

# Dados de treino
X_treino = read_csv('../Dados/dados_tratados/X_treino.csv')
y_treino = read_csv('../Dados/dados_tratados/y_treino.csv').values.ravel()

In [2]:
# Parâmetros
SEMENTE_ALEATORIA = 9 # Random state
num_max_atributos = len(X_treino.columns) - 1

X_treino = X_treino.values

In [3]:
from sklearn.svm import SVC # algoritmo de aprendizado de máquina
from sklearn.preprocessing import StandardScaler, MinMaxScaler # normalização
from sklearn.feature_selection import SequentialFeatureSelector # seleção de atributos
from sklearn.decomposition import PCA # redução de dimensionalidade
from sklearn.pipeline import make_pipeline

def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.

    """
    parametros = {
        "C": trial.suggest_float("C_coef", 0.001, 10),
        "kernel": trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]),
        "gamma": trial.suggest_categorical("fit_hiperplano", ["scale", "auto"]),
        "coef0": trial.suggest_float("coef0", 0, 10),
        "degree": trial.suggest_int("grau_polinomial", 1, 5),
        "max_iter": -1,
        "class_weight": "balanced", # Dados desbalanceados
    }

    normalizacao = trial.suggest_categorical("normalizacao", ["standard", "minmax"])
    usar_pca = trial.suggest_categorical("PCA", [True, False])
    usar_sfs = trial.suggest_categorical("SFS", [True, False])

    num_atributos = trial.suggest_int("num_atributos", 2, num_max_atributos) # para RFE
    num_dimensoes = trial.suggest_int("num_dimensoes", 2, num_atributos) # para PCA
        
    
    steps = []

    if normalizacao == "standard":
        steps.append(("normalizador", StandardScaler()))
    else:
        steps.append(("normalizador", MinMaxScaler()))

    if usar_sfs:
        steps.append(("sfs", SequentialFeatureSelector(
            SVC(**parametros), 
            n_features_to_select=num_atributos,
            direction="backward",
            scoring=f2_score,
            cv=5,
            n_jobs=1,
        )))

    if usar_pca:
        steps.append(("pca", PCA(n_components=num_dimensoes, random_state=SEMENTE_ALEATORIA)))

    steps.append(("modelo", SVC(**parametros)))

    modelo = make_pipeline(*[s[1] for s in steps])

    return modelo

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
import numpy as np

f2_score = make_scorer(fbeta_score, beta=2)

def funcao_objetivo(trial, X, y, num_folds):
    """Função objetivo do optuna.

    Faz validação cruzada estratificada com métrica F2.

    """
    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring=f2_score,
        cv=num_folds,
    )

    return np.mean(metricas)

In [5]:
from optuna import create_study

NOME_DO_ESTUDO = "SVC_optuna"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-03 16:14:53,577] A new study created in RDB with name: SVC_optuna


In [6]:
# Parâmetros
NUM_FOLDS = 10 # Validação cruzada
NUM_TENTATIVAS = 200 # Quantidade de trials

def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

# Faz a otimização
objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 16:14:54,309] Trial 0 finished with value: 0.857341731889021 and parameters: {'C_coef': 6.3032655076203605, 'kernel': 'linear', 'fit_hiperplano': 'scale', 'coef0': 1.382933466562103, 'grau_polinomial': 2, 'normalizacao': 'minmax', 'PCA': False, 'SFS': False, 'num_atributos': 7, 'num_dimensoes': 2}. Best is trial 0 with value: 0.857341731889021.
[I 2025-11-03 16:15:43,058] Trial 1 finished with value: 0.5296277697570775 and parameters: {'C_coef': 4.530419040746595, 'kernel': 'sigmoid', 'fit_hiperplano': 'scale', 'coef0': 8.694688607719169, 'grau_polinomial': 4, 'normalizacao': 'minmax', 'PCA': True, 'SFS': True, 'num_atributos': 3, 'num_dimensoes': 2}. Best is trial 0 with value: 0.857341731889021.
[I 2025-11-03 16:15:44,175] Trial 2 finished with value: 0.8022837692862039 and parameters: {'C_coef': 1.6536834024947897, 'kernel': 'linear', 'fit_hiperplano': 'scale', 'coef0': 8.603584464877635, 'grau_polinomial': 2, 'normalizacao': 'standard', 'PCA': True, 'SFS': False, 'num

In [7]:
df_analysis = objeto_de_estudo.trials_dataframe()

df_analysis

,number,value,datetime_start,datetime_complete,duration,params_C_coef,params_PCA,params_SFS,params_coef0,params_fit_hiperplano,params_grau_polinomial,params_kernel,params_normalizacao,params_num_atributos,params_num_dimensoes,state
0,0,0.857342,2025-11-03 16:14:53.614241,2025-11-03 16:14:54.272845,0 days 00:00:00.658604,6.303266,False,False,1.382933,scale,2,linear,minmax,7,2,COMPLETE
1,1,0.529628,2025-11-03 16:14:54.328573,2025-11-03 16:15:42.922598,0 days 00:00:48.594025,4.530419,True,True,8.694689,scale,4,sigmoid,minmax,3,2,COMPLETE
2,2,0.802284,2025-11-03 16:15:43.079224,2025-11-03 16:15:44.038928,0 days 00:00:00.959704,1.653683,True,False,8.603584,scale,2,linear,standard,6,2,COMPLETE
3,3,0.860733,2025-11-03 16:15:44.192109,2025-11-03 16:16:15.561560,0 days 00:00:31.369451,8.538914,False,True,1.665079,auto,4,poly,minmax,2,2,COMPLETE
4,4,0.833373,2025-11-03 16:16:15.628502,2025-11-03 16:16:32.004009,0 days 00:00:16.375507,6.915972,True,False,8.663354,auto,3,poly,standard,6,5,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,0.854445,2025-11-03 17:58:24.854120,2025-11-03 17:59:29.498621,0 days 00:01:04.644501,4.240471,True,True,9.999640,scale,2,linear,standard,4,4,COMPLETE
196,196,0.869024,2025-11-03 17:59:29.554086,2025-11-03 18:00:14.403585,0 days 00:00:44.849499,4.976198,True,True,3.595579,scale,2,linear,standard,7,6,COMPLETE
197,197,0.866009,2025-11-03 18:00:14.505872,2025-11-03 18:00:56.192033,0 days 00:00:41.686161,4.431546,True,True,9.063771,scale,2,linear,standard,7,6,COMPLETE
198,198,0.869024,2025-11-03 18:00:56.251928,2025-11-03 18:01:34.887177,0 days 00:00:38.635249,4.067685,True,True,9.368742,scale,2,linear,standard,7,6,COMPLETE


In [8]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 82
Parâmetros do melhor trial: {'C_coef': 5.244485922321054, 'kernel': 'linear', 'fit_hiperplano': 'scale', 'coef0': 8.858887342374125, 'grau_polinomial': 2, 'normalizacao': 'standard', 'PCA': True, 'SFS': True, 'num_atributos': 7, 'num_dimensoes': 6}


## Referências

[1] Documentação do optuna. Acesso em: 31/10/2025. Disponível em: https://optuna.readthedocs.io/en/stable/ 

[2] Sperat, Walter. *Using optuna with sklearn the right way*. Acesso em 31/10/2025. Disponível em: https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

[3] Página do Support Vector Machine no scikit-learn. Acesso em: 31/10/2025. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

[4] Página do Support Vector Machine em classificação (SVC) na documentação do scikit-learn. Acesso em: 31/10/2025. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

[5] Helmy, Bahaa; Aibin, Michal. *The C Parameter in Support Vector Machines*. Acesso em: 01/11/2025. Disponível em: https://www.baeldung.com/cs/ml-svm-c-parameter